# Evolution of Disk and Bulge S&eacute;rsic Profiles During the MW-M31 Major Merger

My research project involves examing how the S&eacute;rsic profiles/S&eacute;rsic indices of the bulges and disks of the Milky Way and Andromeda (M31) galaxies evolve throughout their simulated future merger.

In [7]:
# Load Modules
import numpy as np
import astropy.units as u

# import plotting modules
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import rc

# high quality figures
plt.rcParams['savefig.dpi'] = 1200

# Computer Modern with TeX
rc('text', usetex=True)
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from MassProfile import MassProfile
from GalaxyMass import ComponentMass

# garbage collection
import gc

## S&eacute;rsic Profiles

Describe intensity (power per unit area) as a function of cylindrical radius for galaxies.

## ``sersic`` Function : 

We have a function called `sersic` that returns the S&eacute;rsic Profile in terms of the effective radius $R_\mathrm{e}$ (i.e. the half light radius).

$$\large I(r) = I_\mathrm{e} e^{-7.67 \left[ (r/R_\mathrm{e})^{1/n} - 1\right]} $$

Where 

$$\large L = 7.2\pi I_\mathrm{e} R_\mathrm{e}^2 $$

We will assume a mass to light ratio for disk and bulge particles of 1, so **this is also the half mass radius**, and so $\Sigma$, the projected area mass density, is nominally equivalent to the intensity $I$.

The function takes as input the radius, $R_e$, $n$ (S&eacute;rsic index) and the total stellar mass $M_\mathrm{tot}$ of the system.

In [ ]:
def sersic(r, R_e, n, M_tot):
    """ Function that computes a Sersic Profile assuming M/L = 1.
    
    PARMETERS
    ---------
        r: `float`
            Distance from the center of the galaxy (kpc)
            
        R_e: `float`
            Effective radius (2D radius that contains 
            half the light) (kpc)
            
        n:  `float`
            Sersic index
            
        M_tot: `float`
            Total stellar mass (Msun)

    RETURNS
    -------
        I: `array of floats`
            the radial intensity profile of the galaxy in Lsun/kpc^2

    """

    # We are assuming M/L = 1, so the total luminosity is:
    lum = M_tot
    
    # the effective intensity is
    I_e = lum / 7.2 / np.pi / R_e**2
    
    # Break down the equation 
    a = (r / R_e)**(1.0/n)
    b = -7.67 * (a-1)
    
    # Intensity
    #I = Ie*np.exp(-7.67*((r/R_e)**(1.0/n)-1.0))
    I = I_e * np.exp(b)
    
    return I

## ``RadialIntensity`` Class:

In [4]:
class RadialIntensity:
    """Creates an object that can compute and plot a radial intensity profile
    for a galaxy."""

    
    def __init__(self, galaxy, snap, res, comp, r_num):
        """
        Initializes a RadialIntensity object, selecting the right data for the 
        given resolution (res), component (comp), and snapshot (snap) for galaxy
        galaxy. Plot colors, limits, and annotations are set based on the selections
        made for galaxy and comp. The filename to read in, the total mass of the
        given galaxy component, and the number of radii to convert into annuli are
        stored.
        
        Inputs:
            
            galaxy : `string`
                Name of the galaxy to create an intensity profile for ('MW' or 'M31')
                (case-sensitive)
                
            snap : `int`
                Snapshot (0–801) to read in.
                
            res : `string`
                'high' or 'low' simulation resolution (case-sensitive)
                
            comp : `string`
                Galaxy component, 'Disk' or 'Bulge' (case-sensitive)
                
            r_num : `int`
                numbr of radii to turn into annuli inner and outer radii for computing
                intensities within.
                
        Returns:
        
            None
        """
        
        # Select the desired simulation resolution
        if res == 'high':
            snap_path = 'HighRes_' + galaxy + '/'
        elif res == 'low':
            snap_path = 'VLowRes_' + galaxy + '/'
        
        # Component options
        if comp == 'Disk':
            self.p_type = 2 # particle type for disk particles
            
            # Differentiate colors by component _and_ galaxy
            if galaxy == 'MW':
                self.color = 'blue'
            else:
                self.color = 'red'
            
            # Set the plot limits for a disk
            self.xlim = (10**(-1), 200)
            self.ylim = (10**2, 10**11)
            
            # Time annotation location in axis space
            self.annotate = (1, 10**7)
    
        elif comp == 'Bulge':
            self.p_type = 3 # particle type for bulge particles
            
            # Differentiate colors by component _and_ galaxy
            if galaxy == 'MW':
                self.color = 'orange'
            else:
                self.color='green'
            
            # Set the plot limits for a bulge
            self.xlim = (10**(-1), 10**3)
            self.ylim = (10**(-3), 10**11)
            
            # Time annotation location in axis space
            self.annotate = (1, 1)
            
        # store the component string for later use when plotting
        self.comp = comp
        
        # add a string of the filenumber to the value “000”
        snap_str= '000' + str(snap)
        # remove all but the last 3 digits
        snap_str = snap_str[-3:]
        
        # Store the snap_str for later use when writing plots to PNGs
        self.snap_str = snap_str

        # construct filename for later use
        self.filename = snap_path + galaxy + '_' + snap_str + '.txt'
        
        # Store the galaxy for later use when plotting
        self.galaxy = galaxy
        # store the snap to compute the time in Gyr later
        self.snap = snap
        # store the number of radii for later annuli generation
        self.r_num = r_num
        
        # Store the total mass for the given component for creating a Sersic profile
        self.m_tot = ComponentMass(self.filename, self.p_type) * 1e12 # Msun


    def profile(self):
        """
        Calculates a radial area mass density profile using annuli in cylindrical
        coordinates.
        
        Inputs: 
        
            None
            
        Returns: 
            
            (r_annuli, I, R_e) : `tuple`
                Tuple of the annuli that area mass densities are calculated within,
                r_annuli, the intensity profile, I, and the effective
                (half-light / half-mass) radius, R_e. These are used for plotting.
        """
        
        # Create a center of mass object
        # This lets us get the x, y, z relative to the COM
        COM = CenterOfMass(self.filename, self.p_type)
        COM_p = COM.COM_P(0.1) # COM position

        # COM.x, COM.y, COM.z, COM.m are arrays
        x = COM.x - COM_p[0].value
        y = COM.y - COM_p[1].value
        z = COM.z - COM_p[2].value
        m = COM.m # array of masses of individual particles of type self.p_type

        # calculate the radial distances of particles in cylindrical coordinates
        cyl_r_mag = np.sqrt(x**2 + y**2)

        # create a logspace (base 2) array of radii so that annuli radii increase
        # as we move out into the more sparse outer regions
        radii = np.logspace(np.log2(0.1), np.log2(0.4*cyl_r_mag.max()),
                            num=self.r_num, base=2) # kpc

        # create the mask to select particles enclosed for each radius
        # np.newaxis creates a virtual axis to make tmp_r_mag 2 dimensional
        # so that all radii can be compared simultaneously
        enc_mask = cyl_r_mag[:, np.newaxis] < np.asarray(radii).flatten()

        # calculate the enclosed masses 
        # relevant particles will be selected by enc_mask (i.e., *1)
        # outer particles will be ignored (i.e., *0)
        m_enc = np.sum(m[:, np.newaxis] * enc_mask, axis=0) * 1e10 # Msun

        # use the difference between nearby elements to get mass in each annulus
        m_annuli = np.diff(m_enc) # one element less then m_enc
        
        # Area mass density
        Sigma = m_annuli / (np.pi * (radii[1:]**2 - radii[:-1]**2))
        
        # convert area mass density to intensity with a mass-to-light ratio (1, here)
        m_to_l = 1
        I = Sigma / m_to_l

        r_annuli = np.sqrt(radii[1:] * radii[:-1]) 
        # here we choose the geometric mean
        
        half_mass = self.m_tot / 2
        indices = np.where(m_enc > half_mass)
        R_maj = radii[indices]

        # the first such index gives us the index of our half-light radius
        R_e = R_maj[0] # kpc

        # return the stuff that we need to make a plot
        return (r_annuli, I, R_e)
    
    
    def plot_profile(self, r_annuli, I, R_e):
        """
        Uses Matplotlib to create PNGs of radial intensity profiles compared with 
        a de Vaucouleurs (Sersic index n = 4) profile.
        
        Inputs:
            
            r_annuli : `numpy array`
                1D array of float average radius for each annulus.
            
            I : `numpy array`
                1D array of float intensity within each annulus
                
            R_e : `float`
                Effective (half-light / half-mass radius) for the galaxy component
                
        Returns:
            
            None
        """
        
        # Set up the plot
        fig, ax = plt.subplots()

        n4 = 4 # de Vaucouleurs Sersic index
        
        # plot a de Vaucouleurs profile (log-log)
        plt.loglog(r_annuli, sersic(r_annuli, R_e, n4, self.m_tot), color='k',
                     linestyle="-.", label=r'S\'{{e}}rsic $n={}$'.format(n4),
                     linewidth=1)

        # Plot the calculated intensity profile (log-log)
        ax.loglog(r_annuli, I, alpha=0.8, label='Simulated '+self.comp,
                  linewidth=2, color=self.color)
        
        fig.set_size_inches(1.90, 1) # approx. DCI 4K aspect ratio
        
        # Axis labels, axis limits, and plot title
        ax.set(xlabel=r'$r \ (\mathrm{kpc})$',
               ylabel=r'$I \ \left(\mathrm{L_\odot} / \mathrm{kpc}^2\right)$',
               title=self.galaxy+' '+ self.comp + ' Particle Radial Intensity Profile',
               xlim=self.xlim, ylim=self.ylim)
        

        # snap / 0.7 = year / 10Myr
        # year = 10 Myr * snap / 0.7
        # Gyr = 0.01 * snap / 0.7
        Gyr = self.snap * 0.01 / 0.7
        # Annotate with the simulation time (0 is the present epoch)
        plt.annotate(r'$\mathrm{{t}} = {:.2f} \ \mathrm{{Gyr}}$'.format(Gyr),
                     self.annotate)

        # legend and tidy things up
        ax.legend(loc='best')
        fig.tight_layout()
        
        # Set the folder to save the plot PNG to
        folder = galaxy + '_' + self.comp + '/'
        
        # Save the plot as a PNG
        plt.savefig(folder + self.galaxy+'_'+self.snap_str+'_'+self.comp+'.png',
                    facecolor='w')
        
        # all this together seems to fix the memory leak ??
        # Clear the current axes
        plt.cla()
        # Clear the figure
        fig.clf()
        # close everything
        plt.close('all')
        # close the figure
        plt.close(fig)
        # ???
        plt.ioff()
        # delete variables
        del r_annuli, I, folder, Gyr
        # collect garbage
        gc.collect()

## Options!

In [1]:
# options
res = 'high' # resolution of data
r_num = 35 # number of radii to become (r_num - 1) annuli

# Choose which snaps to start and stop at, with what snap step
snap_start = 0 # inclusive
snap_end = 401 # exclusive
snap_step = 1

# Choose the galaxy and component
galaxy = 'MW'
component = 'Disk'

## Compute surface brightness profiles and plot

In [6]:
# Loop over snapshots, get and plot surface brightness profiles using the SurfaceBrightness class
for snap in range(snap_start, snap_end, snap_step):

    # Initialize the class
    radial_intensity = RadialIntensity(galaxy, snap, res, component, r_num)

    # Get the radii, profile, and equivalent radius for the file used
    *plot_params ,= radial_intensity.profile()

    # plot
    radial_intensity.plot_profile(*plot_params)
    
    # more memory leak management
    # delete variables
    del radial_intensituy, plot_params
    # collect garbage
    gc.collect()

## Next steps:
1) Fit S&eacute;rsic profiles with ``scipy.optimize.curvefit()``, instead of just plotting an $n = 4$ profile.
2) Analyze how the fit changes through the simulation, particularly where it changes the most (after pericenters?)
3) Check out other cool effects (waves? Standing/traveling?)